In [ ]:
%%writefile hatem_figure_functions.py

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
import matplotlib.cm as cm
from scipy.ndimage import binary_dilation, binary_fill_holes

def getMaxImg(ops):
    maxImg = ops["max_proj"] # Also "max_proj", "meanImg", "meanImgE"
    mimg = maxImg # Use suite-2p source-code naming
    mimg1 = np.percentile(mimg,1)
    mimg99 = np.percentile(mimg,99)
    mimg = (mimg - mimg1) / (mimg99 - mimg1)
    mimg = np.maximum(0,np.minimum(1,mimg))
    mimg *= 255
    mimg = mimg.astype(np.uint8)
    return mimg

def boundary(ypix,xpix):
    """ returns pixels of mask that are on the exterior of the mask
        Imported from suite2p.gui.utils directly because I cannot yet navigate directories in Python """
    ypix = np.expand_dims(ypix.flatten(),axis=1)
    xpix = np.expand_dims(xpix.flatten(),axis=1)
    npix = ypix.shape[0]
    if npix>0:
        msk = np.zeros((np.ptp(ypix)+6, np.ptp(xpix)+6), np.bool)
        msk[ypix-ypix.min()+3, xpix-xpix.min()+3] = True
        msk = binary_dilation(msk)
        msk = binary_fill_holes(msk)
        k = np.ones((3,3),dtype=int) # for 4-connected
        k = np.zeros((3,3),dtype=int); k[1] = 1; k[:,1] = 1 # for 8-connected
        out = binary_dilation(msk==0, k) & msk

        yext, xext = np.nonzero(out)
        yext, xext = yext+ypix.min()-3, xext+xpix.min()-3
    else:
        yext = np.zeros((0,))
        xext = np.zeros((0,))
    return yext, xext

def getStats(stat, frame_shape):
    min_skew = 1 #TODO: define for iscell; also figure exclude noncell
    max_pixel = 200
    max_compact = 1.25
    min_footprint = 0
    max_footprint = 3
    pixel2neuron = np.full((sum(stat.shape), sum(stat.shape)), fill_value=np.nan, dtype=float)
    scatters = dict(x=[], y=[], color=[], text=[])
    min_skew, max_skew = -20000, 20000 #swapoped min/max 20,000 *-1
    nid2idx = {}
    nid2idx_rejected = {}
    print(f"Num cells: {stat.shape[0]}")
    # from suite2p.gui.utils import boundary; recreated the function manually
    for n in range(stat.shape[0]):
        skew = stat[n]['skew']
        npix = stat[n]['npix']
        footprint = stat[n]['footprint']
        compact = stat[n]['compact']
        min_skew, max_skew = min(skew, min_skew), max(skew, max_skew)
        if skew >= min_skew and npix <= max_pixel and compact <= max_compact and footprint >= min_footprint and footprint <= max_footprint:
            nid2idx[n] = len(scatters["x"]) # Assign new idx
        else:
            nid2idx_rejected[n] = len(scatters["x"])
        ypix = stat[n]['ypix'].flatten() - 1#[~stat[n]['overlap']] - 1
        xpix = stat[n]['xpix'].flatten() - 1#[~stat[n]['overlap']] - 1
        yext, xext = boundary(ypix, xpix)
        scatters['x'] += [xext]
        scatters['y'] += [yext]
        pixel2neuron[xpix, ypix] = n
        scatters["color"].append(skew)
        scatters["text"].append(f"Cell #{n} - Skew: {skew}")
    print("Min/max skew: ", min_skew, max_skew)
    # Normalize colors between 0 and 1 ---> not really necessary since its just a sanity check
    # color_raw = np.array(scatters["color"])
    # scatters["color"] = (color_raw - min_skew) / (max_skew - min_skew)
    return scatters, nid2idx, nid2idx_rejected, pixel2neuron



def dispPlot(MaxImg, scatters, nid2idx, nid2idx_rejected,
             pixel2neuron, F, Fneu, axs=None):
             "Compiles previous functions to produce an image with outlined ROIs outside of suite2p GUI"
  if axs is None:
    fig = plt.figure(constrained_layout=True)
    NUM_GRIDS=12
    gs = fig.add_gridspec(NUM_GRIDS, 1)
    ax1 = fig.add_subplot(gs[:NUM_GRIDS-2])
    # ax2 = fig.add_subplot(gs[NUM_GRIDS-2])
    fig.set_size_inches(12,14)
  else:
    ax1 = axs
    ax1.set_xlim(0, MaxImg.shape[0])
    ax1.set_ylim(MaxImg.shape[1], 0)
    # ax2 = None
  ax1.imshow(MaxImg, cmap='gist_gray')
  #fig.update_xaxes(showticklabels=False)
  #fig.update_yaxes(showticklabels=False)
  print("Neurons count:", len(nid2idx))
  norm = matplotlib.colors.Normalize(vmin=0, vmax=1, clip=True)
  mapper = cm.ScalarMappable(norm=norm, cmap=cm.plasma)

  def plotDict(n2d2idx_dict, override_color = None):
    for neuron_id, idx in n2d2idx_dict.items():
      color = override_color if override_color else mapper.to_rgba(scatters['color'][idx])
    # print(f"{idx}: {scatters['x']} - {scatters['y'][idx]}")
      
      sc = ax1.scatter(scatters["x"][idx], scatters['y'][idx], color = color, 
                     marker='.', s=1)
  plotDict(nid2idx)
  plotDict(nid2idx_rejected, 'k')
  ax1.set_title(f"Neurons Used: {len(nid2idx)}/{len(nid2idx)+len(nid2idx_rejected)} detected (orange)") #pontentially change the color of detected/reject from black to red??
  # if ax2 is None:
  #   return
  # nan_fills = np.full(F.shape[1], fill_value = np.nan)
  # neuron_trace_plot = ax2.plot(np.arange(F.shape[1]), nan_fills, color = 'blue')[0]
  # neuron_trace_plot2 = ax2.plot(np.arange(F.shape[1]), nan_fills, color = 'red')[0]
  # text = ax1.text(0,1.06, "No neurons selected")
  # print(f"spks.shape: {spks.dtype}") --> unnecessary since spks are calculated manually

In [1]:
def boundary(ypix,xpix):
    """ returns pixels of mask that are on the exterior of the mask """
    ypix = np.expand_dims(ypix.flatten(),axis=1)
    xpix = np.expand_dims(xpix.flatten(),axis=1)
    npix = ypix.shape[0]
    if npix>0:
        msk = np.zeros((np.ptp(ypix)+6, np.ptp(xpix)+6), np.bool)
        msk[ypix-ypix.min()+3, xpix-xpix.min()+3] = True
        msk = binary_dilation(msk)
        msk = binary_fill_holes(msk)
        k = np.ones((3,3),dtype=int) # for 4-connected
        k = np.zeros((3,3),dtype=int); k[1] = 1; k[:,1] = 1 # for 8-connected
        out = binary_dilation(msk==0, k) & msk

        yext, xext = np.nonzero(out)
        yext, xext = yext+ypix.min()-3, xext+xpix.min()-3
    else:
        yext = np.zeros((0,))
        xext = np.zeros((0,))
    return yext, xext
